In [1]:
import pandas as pd
import numpy as np
import math

In [2]:
tc_df = pd.read_csv('Data/Timber Contract Data.csv')

# Data Cleaning

In [3]:
#only keeping certain variables
variables_to_keep = ['Contract Identifier','State Office Code','Contract Term','Acres Current','Mbf Current','Ccf Current','Value Sold']
tc_df_subset = tc_df[variables_to_keep]

In [4]:
#Taking out the rows for which 'Value Sold' is 0
tc_df_subset = tc_df_subset[tc_df_subset['Value Sold']!=0]
#Taking out entries where value sold is nan
tc_df_subset = tc_df_subset[tc_df_subset['Value Sold'].notnull()]
#Reindexing for later use
tc_df_subset = tc_df_subset.reset_index(drop=True)

In [5]:
#Creating a log showing where variables are either 'nan' or blank
list_of_lists = []
log = []*len(variables_to_keep)
for i in range(len(variables_to_keep)):
    list_of_lists.append(tc_df_subset[variables_to_keep[i]])
for i in range(len(variables_to_keep)):
    temp = []
    for j in range(len(list_of_lists[i])):
        if str(list_of_lists[i][j]) == "" or str(list_of_lists[i][j])=="nan":
            temp.append([variables_to_keep[i],j,str(list_of_lists[i][j])])
    log.append(temp)

In [6]:
#going through and recording the indeces for which Mbf an CCf terms are either blank or nan
Mbf_terms = []
Ccf_terms = []
for i in range(len(log[4])):
    Mbf_terms.append(log[4][i][1])
for i in range(len(log[5])):
    Ccf_terms.append(log[5][i][1])

#check to see if the terms are mostly overlapping
check_mbf = []
for i in Mbf_terms:
    if i in Ccf_terms:
        check_mbf.append(0)
    else:
        check_mbf.append(1)
check_ccf = []
for i in Ccf_terms:
    if i in Mbf_terms:
        check_ccf.append(0)
    else:
        check_ccf.append(1)

print(sum(check_mbf))
print(len(check_mbf))
print(sum(check_ccf))
print(len(check_ccf))

11
101
0
90


In [7]:
#this shows that the terms with blank or nan or mostly overlapping which is good and will mean less loss of data
#I will go ahead and delete these rows from the dataframe and then reindex
indeces_to_delete = []
for i in Mbf_terms:
    indeces_to_delete.append(i)
for i in Ccf_terms:
    if i not in indeces_to_delete:
        indeces_to_delete.append(i)
tc_df_subset = tc_df_subset.drop(indeces_to_delete)
tc_df_subset = tc_df_subset.reset_index(drop=True)

In [8]:
#---------------treatment of timber contracts dataframe----------------------------
#now the first goal is going to be getting out one row of data per unique contract identifier
#since each quarter provides an update for all contracts not terminated, many contracts will be listed more than once
#Also, the factors associated with them will be changing as the logging process incurrs
#So for items like Acres Current and Mbf Current and CCf Current, these values will mostly be decreasing as time goes by and more
#trees have been felled. However, in some instances it will be increasing as sometimes the original estimates of MBF and CCF were too low
#and the estimates go up. Therefore, to get the original estimate of acres, MBF, and CCF I will only keep the bottom row for
#each unique contract identifier

unique_contract_identifiers = np.unique(tc_df_subset['Contract Identifier'])

indeces_to_keep = []
for contractid in unique_contract_identifiers:
    new_subset = tc_df_subset[tc_df_subset['Contract Identifier']==contractid]
    new_subset_indeces = new_subset.index
    indeces_to_keep.append(new_subset_indeces[-1])
    
new_tc_df = tc_df_subset.loc[indeces_to_keep,:]

new_tc_df = new_tc_df.reset_index(drop=True)

# Standardizing and Scaling Features

In [9]:
#function that takes in a variable as a pandas series and returns the standardized values in a pandas series. 
#Standardization is done using the mean and standard deviation
def standardize(variable):
    mean = sum(variable)/len(variable)
    total_var = 0
    for i in variable:
        total_var = total_var + (i-mean)*(i-mean)
    std = math.sqrt(total_var/len(variable))
    
    standardized_variable = []
    for i in variable:
        standardized_variable.append((i-mean)/std)
    return pd.Series(standardized_variable)

In [10]:
new_tc_df['Value Sold Standardized'] = standardize(new_tc_df['Value Sold'])
new_tc_df['Acres Current Standardized'] = standardize(new_tc_df['Acres Current'])
new_tc_df['Mbf Current Standardized'] = standardize(new_tc_df['Mbf Current'])
new_tc_df['Ccf Current Standardized'] = standardize(new_tc_df['Ccf Current'])

In [11]:
new_tc_df['Value Sold sqrt standardized'] = standardize(np.sqrt(new_tc_df['Value Sold']))
new_tc_df['Value Sold log standardized'] = standardize(np.log(new_tc_df['Value Sold']))

# Engineered Features

In [12]:
#creating a feature for the combination of Ccf and Mbf
new_tc_df['Ccf Mbf'] = (new_tc_df['Mbf Current Standardized']+new_tc_df['Ccf Current Standardized'])/2

In [13]:
#creating a feature for the combination of the the standardized interactions between Acres and Ccf and Acres and Mbf
acres_ccf_standardized = standardize(new_tc_df['Acres Current']*new_tc_df['Ccf Current'])
acres_mbf_standardized = standardize(new_tc_df['Acres Current']*new_tc_df['Mbf Current'])
new_tc_df['Acre Volume'] = (acres_ccf_standardized+acres_mbf_standardized)/2

# Encoding Qualitative Features

In [14]:
#creates dummy variables for the qualitative features. takes in pandas series and returns a pandas dataframe of the
#dummy variables.
def encodeQualitative(variable):
    unique_values = np.unique(variable)
    new_vars = []
    for i in range(len(variable)):
        new_line = []
        for j in range(len(unique_values)-1):
            if variable[i] == unique_values[j]:
                new_line.append(1)
            else:
                new_line.append(0)
        new_vars.append(new_line)
    new_vars_df = pd.DataFrame(new_vars,columns=unique_values[:-1])
    return new_vars_df

In [15]:
new_state_vars = encodeQualitative(new_tc_df['State Office Code'])
new_tc_df = pd.concat([new_tc_df,new_state_vars],axis=1)

# Interaction Terms

In [16]:
#making an interaction term between Ccf Mbf and each state dummy variable
state_columns = new_state_vars.columns
new_state_columns = []
state_interactions = []
#creating the state interaction labels
for c in state_columns:
    new_state_columns.append(c+" Ccf Mbf")
#creating the interaction terms
for i in range(len(new_tc_df['Ccf Mbf'])):
    state_interactions.append(list(new_state_vars.loc[i,:]*new_tc_df.loc[i,'Ccf Mbf']))
    
new_state_interactions = pd.DataFrame(state_interactions,columns=new_state_columns)
new_tc_df = pd.concat([new_tc_df,new_state_interactions],axis=1)

# Writing Cleaned Data Set

In [17]:
#now I will write this data to a new csv file that will be uploaded for the next sections - visualizations and analysis
new_tc_df.to_csv('Cleaned Timber Data.csv',index=False)